# **IMPORT LIBRARY**

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set_theme()
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler 

# **IMPORT DATA**

In [4]:
# Import data from github
data = pd.read_excel(r"C:\Users\ASUS\Desktop\Science Research (NCKH)\Processing\Predict Ethereum price movement by News and Blockchain information\pred_eth_data.xlsx", index_col=0, parse_dates=True)
data.index = data.index.astype('datetime64[ns]')
data

,Negative,Positive,return_bitcoin,return_litecoin,avg_diff,act_address,avg_gas,trans_count,block_count,block_size,etherium
Time,,,,,,,,,,,
2017-11-10,0,0,-0.073554,-0.077946,1430000000000000,233966,0.1911,457427,6300,84500680,0.051555
2017-11-11,0,1,-0.039368,0.051353,1440000000000000,194696,0.1867,394378,6267,79370301,-0.021523
2017-11-12,0,0,-0.064101,-0.052933,1490000000000000,213315,0.2106,434380,6245,82639583,0.028606
2017-11-13,0,1,0.102422,0.040523,1460000000000000,214384,0.2203,443176,6225,89907928,0.066037
2017-11-14,0,0,0.011626,0.020899,1450000000000000,226170,0.2359,475984,6122,99374433,-0.012659
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-07,0,1,-0.014124,-0.036100,11088900000000000,517421,2.7500,972121,7075,550842346,0.039498
2022-12-08,1,0,0.022872,0.017574,11088900000000000,528927,2.7300,923887,7161,512401884,-0.013138
2022-12-09,0,1,-0.005821,-0.026219,11088900000000000,1505652,2.3900,1932711,7152,585484918,0.001661


In [5]:
# Check NaN values ​​in dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1856 entries, 2017-11-10 to 2022-12-11
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Negative         1856 non-null   int64  
 1   Positive         1856 non-null   int64  
 2   return_bitcoin   1856 non-null   float64
 3   return_litecoin  1856 non-null   float64
 4   avg_diff         1856 non-null   int64  
 5   act_address      1856 non-null   int64  
 6   avg_gas          1856 non-null   float64
 7   trans_count      1856 non-null   int64  
 8   block_count      1856 non-null   int64  
 9   block_size       1856 non-null   int64  
 10  etherium         1856 non-null   float64
dtypes: float64(4), int64(7)
memory usage: 174.0 KB


In [6]:
data.Negative, data.Positive = data.Negative.astype('object'), data.Positive.astype('object')
discrete_data = data[['Negative','Positive']]
discrete_data.describe()


,Negative,Positive
count,1856,1856
unique,2,2
top,0,1
freq,1519,1400


# **DESCRIPTIVE STATISTICS**  

In [7]:
dt = data.describe().T
dt.to_excel(r"C:\Users\ASUS\Desktop\Science Research (NCKH)\Processing\Predict Ethereum price movement by News and Blockchain information\des_statis.xlsx")

In [ ]:
dt

# **BUILDING MODEL**

In [9]:
df = data.loc['2017-11-10':'2021-11-10']
df = data.copy()
df.index.set_names('Train sheet',inplace=True)
# df.drop(["return_etherium"],axis=1,inplace=True)

In [10]:
 def cag_eth(score):
  if score <= 0:
    return 0
  elif score > 0:
    return 1

df["return_etherium"] = df['etherium'].apply(cag_eth)
df.drop(["etherium"],axis=1,inplace=True)

In [11]:
df

,Negative,Positive,return_bitcoin,return_litecoin,avg_diff,act_address,avg_gas,trans_count,block_count,block_size,return_etherium
Train sheet,,,,,,,,,,,
2017-11-10,0,0,-0.073554,-0.077946,1430000000000000,233966,0.1911,457427,6300,84500680,1
2017-11-11,0,1,-0.039368,0.051353,1440000000000000,194696,0.1867,394378,6267,79370301,0
2017-11-12,0,0,-0.064101,-0.052933,1490000000000000,213315,0.2106,434380,6245,82639583,1
2017-11-13,0,1,0.102422,0.040523,1460000000000000,214384,0.2203,443176,6225,89907928,1
2017-11-14,0,0,0.011626,0.020899,1450000000000000,226170,0.2359,475984,6122,99374433,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-07,0,1,-0.014124,-0.036100,11088900000000000,517421,2.7500,972121,7075,550842346,1
2022-12-08,1,0,0.022872,0.017574,11088900000000000,528927,2.7300,923887,7161,512401884,0
2022-12-09,0,1,-0.005821,-0.026219,11088900000000000,1505652,2.3900,1932711,7152,585484918,1


In [12]:
# Set the independent variable property
feature = df.columns[df.columns!='return_etherium'].to_list()
target=  [df.columns[-1]]

In [13]:
y = df[target]
X = df[feature]

# Split the train and test
n_state = 45
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state=n_state)

In [14]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

**Linear Logistic**

In [16]:
from sklearn.linear_model import LogisticRegression
LR_classifier = LogisticRegression()
LR_classifier.fit(X_train, y_train)

y_pred = LR_classifier.predict(X_test)


# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_test,y_pred)}')

# Tỉ lệ chính xác accuracy
print('Logistic Regression accuracy: ', round(accuracy_score(y_test, y_pred),4))



Confusion Matrix:
[[ 77 107]
 [ 44 144]]

Report model:
              precision    recall  f1-score   support

           0       0.64      0.42      0.50       184
           1       0.57      0.77      0.66       188

    accuracy                           0.59       372
   macro avg       0.61      0.59      0.58       372
weighted avg       0.60      0.59      0.58       372

Logistic Regression accuracy:  0.5941


In [17]:
import statsmodels.api as SM 
table = SM.Logit(y_train,X_train).fit()
print(table.summary())

Optimization terminated successfully.
         Current function value: 0.673335
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:        return_etherium   No. Observations:                 1484
Model:                          Logit   Df Residuals:                     1474
Method:                           MLE   Df Model:                            9
Date:                Mon, 20 Mar 2023   Pseudo R-squ.:                 0.02802
Time:                        23:29:42   Log-Likelihood:                -999.23
converged:                       True   LL-Null:                       -1028.0
Covariance Type:            nonrobust   LLR p-value:                 3.854e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.3709      0.099     -3.734      0.000      -0.566      -0.176
x2            -0.0688      0.

**Randome Forest**

In [18]:
# Xây dựng mô hình Random Forest
RF_classifier = RandomForestClassifier(random_state=n_state)

RF_classifier.fit(X_train, y_train)

# Dự báo giá trị biến phụ thuộc
y_pred = RF_classifier.predict(X_test)
y_pred = pd.DataFrame(y_pred,index=y_test.index,columns=['Random Forest Prediction'])


# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_test,y_pred)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_test, y_pred),4))

Confusion Matrix:
[[ 88  96]
 [ 71 117]]

Report model:
              precision    recall  f1-score   support

           0       0.55      0.48      0.51       184
           1       0.55      0.62      0.58       188

    accuracy                           0.55       372
   macro avg       0.55      0.55      0.55       372
weighted avg       0.55      0.55      0.55       372

Random Forest accuracy:  0.5511


In [19]:
# Xây dựng mô hình Random Forest
RF_classifier = RandomForestClassifier(random_state=n_state,
                                        n_estimators=2000, 
                                        min_samples_leaf = 2,
                                        max_features = "log2", 
                                        max_depth = 15,
                                        min_samples_split=2,
                                        criterion= "log_loss",
                                        warm_start= True,
                                        class_weight = {0:4,
                                                        1:6})

RF_classifier.fit(X_train, y_train)

# Dự báo giá trị biến phụ thuộc
y_pred = RF_classifier.predict(X_test)
y_pred = pd.DataFrame(y_pred,index=y_test.index,columns=['Random Forest Prediction'])


# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_test,y_pred)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_test, y_pred),4))

Confusion Matrix:
[[ 64 120]
 [ 44 144]]

Report model:
              precision    recall  f1-score   support

           0       0.59      0.35      0.44       184
           1       0.55      0.77      0.64       188

    accuracy                           0.56       372
   macro avg       0.57      0.56      0.54       372
weighted avg       0.57      0.56      0.54       372

Random Forest accuracy:  0.5591


In [20]:
# result_RF = pd.concat([y_test,y_pred],axis=1)
# result_RF.sort_index(inplace=True)
# result_RF.head(20)

In [21]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(n_neighbors = 30),max_samples=0.5, max_features=0.5)

bagging.fit(X_train, y_train)

# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_test,y_pred)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_test, y_pred),4))

Confusion Matrix:
[[ 64 120]
 [ 44 144]]

Report model:
              precision    recall  f1-score   support

           0       0.59      0.35      0.44       184
           1       0.55      0.77      0.64       188

    accuracy                           0.56       372
   macro avg       0.57      0.56      0.54       372
weighted avg       0.57      0.56      0.54       372

Random Forest accuracy:  0.5591


**Support Vector Regression**

In [22]:
from sklearn.svm import SVC
classifier_lr = SVC(kernel = 'linear', probability=True, random_state = 45)
classifier_lr.fit(X_train, y_train)

y_pred = classifier_lr.predict(X_test)

print('Confusion matrix:')
print(confusion_matrix(y_test,y_pred),'\n')

print('Classification report:')
print(classification_report(y_test,y_pred))

print('Support Vector Machine accuracy: ', round(accuracy_score(y_test, y_pred),4))

Confusion matrix:
[[ 44 140]
 [ 16 172]] 

Classification report:
              precision    recall  f1-score   support

           0       0.73      0.24      0.36       184
           1       0.55      0.91      0.69       188

    accuracy                           0.58       372
   macro avg       0.64      0.58      0.52       372
weighted avg       0.64      0.58      0.53       372

Support Vector Machine accuracy:  0.5806


In [23]:
from sklearn.svm import SVC
classifier_rbf = SVC(kernel = 'rbf', probability=True, random_state = 45)
classifier_rbf.fit(X_train, y_train)

y_pred = classifier_rbf.predict(X_test)

print('Confusion matrix:')
print(confusion_matrix(y_test,y_pred),'\n')

print('Classification report:')
print(classification_report(y_test,y_pred))

print('Support Vector Machine accuracy: ', round(accuracy_score(y_test, y_pred),4))

Confusion matrix:
[[ 65 119]
 [ 39 149]] 

Classification report:
              precision    recall  f1-score   support

           0       0.62      0.35      0.45       184
           1       0.56      0.79      0.65       188

    accuracy                           0.58       372
   macro avg       0.59      0.57      0.55       372
weighted avg       0.59      0.58      0.55       372

Support Vector Machine accuracy:  0.5753


In [24]:
from sklearn.svm import SVC
classifier_poly = SVC(kernel = 'poly', probability=True, random_state = 45)
classifier_poly.fit(X_train, y_train)

y_pred = classifier_poly.predict(X_train)

print('Confusion matrix:')
print(confusion_matrix(y_train,y_pred),'\n')

print('Classification report:')
print(classification_report(y_train,y_pred))

print('Support Vector Machine accuracy: ', round(accuracy_score(y_train, y_pred),4))

Confusion matrix:
[[241 480]
 [117 646]] 

Classification report:
              precision    recall  f1-score   support

           0       0.67      0.33      0.45       721
           1       0.57      0.85      0.68       763

    accuracy                           0.60      1484
   macro avg       0.62      0.59      0.57      1484
weighted avg       0.62      0.60      0.57      1484

Support Vector Machine accuracy:  0.5977


In [67]:
from sklearn.svm import SVC
classifier_poly = SVC(kernel = 'linear', probability=True, random_state = 45)
classifier_poly.fit(X_train, y_train)

y_pred = classifier_poly.predict(X_test)

print('Confusion matrix:')
print(confusion_matrix(y_test,y_pred),'\n')

print('Classification report:')
print(classification_report(y_test,y_pred))

print('Support Vector Machine accuracy: ', round(accuracy_score(y_test, y_pred),4))

Confusion matrix:
[[ 44 140]
 [ 16 172]] 

Classification report:
              precision    recall  f1-score   support

           0       0.73      0.24      0.36       184
           1       0.55      0.91      0.69       188

    accuracy                           0.58       372
   macro avg       0.64      0.58      0.52       372
weighted avg       0.64      0.58      0.53       372

Support Vector Machine accuracy:  0.5806


# **PREDICT ON THE NEW DATASET**

**Import new dataset**

In [26]:
# Call new dataset
df_new = data.loc['2021-11-11':'2022-05-29']
df_new.index = df_new.index.set_names('Test sheet')
# df_new.drop(["return_etherium"],axis=1,inplace=True)

df_new["return_etherium"] = df_new['etherium'].apply(cag_eth)
df_new.drop(["etherium"],axis=1,inplace=True)
df_new

,Negative,Positive,return_bitcoin,return_litecoin,avg_diff,act_address,avg_gas,trans_count,block_count,block_size,return_etherium
Test sheet,,,,,,,,,,,
2021-11-11,0,1,-0.000697,0.006980,11000000000000000,648382,56.8163,1347643,6373,407898241,0
2021-11-12,0,0,-0.012225,-0.043820,10900000000000000,643078,52.2226,1345131,6421,397037354,0
2021-11-13,0,1,0.004888,0.025977,11200000000000000,645915,47.0661,1300464,6326,408939073,0
2021-11-14,0,1,0.015470,0.077159,11000000000000000,643454,41.1622,1275744,6345,411628527,0
2021-11-15,0,1,-0.029159,-0.054833,11200000000000000,606036,54.6244,1289636,6319,431699193,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-25,0,1,-0.003144,-0.028561,14400000000000000,508395,6.3254,1094639,6165,633978416,0
2022-05-26,0,1,-0.009984,-0.065653,14200000000000000,474790,8.0630,1071180,6092,687245210,0
2022-05-27,1,0,-0.021856,-0.035474,14600000000000000,494940,6.8163,1093469,6218,618243706,1


In [27]:
# Set the independent variable property
feature_new = df_new.columns[df_new.columns!='return_etherium'].to_list()
target_new =  [df_new.columns[-1]]

y_new = df_new[target_new]
X_new = df_new[feature_new]

In [28]:
sc_X_new = StandardScaler()

# Scale train set
X_new = sc_X_new.fit_transform(X_new)

**Randome Forest**


In [29]:
# Dự báo giá trị biến phụ thuộc
y_new_pred = RF_classifier.predict(X_new)
y_new_pred = pd.DataFrame(y_new_pred,index=y_new.index,columns=['Random Forest Prediction'])


# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_new,y_new_pred)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_new,y_new_pred)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_new, y_new_pred),4))

Confusion Matrix:
[[29 75]
 [18 78]]

Report model:
              precision    recall  f1-score   support

           0       0.62      0.28      0.38       104
           1       0.51      0.81      0.63        96

    accuracy                           0.54       200
   macro avg       0.56      0.55      0.51       200
weighted avg       0.57      0.54      0.50       200

Random Forest accuracy:  0.535


**Support Vector Regression**

In [30]:
y_new_pred = classifier_lr.predict(X_new)

# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_new,y_new_pred)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_new,y_new_pred)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_new, y_new_pred),4))

Confusion Matrix:
[[21 83]
 [ 9 87]]

Report model:
              precision    recall  f1-score   support

           0       0.70      0.20      0.31       104
           1       0.51      0.91      0.65        96

    accuracy                           0.54       200
   macro avg       0.61      0.55      0.48       200
weighted avg       0.61      0.54      0.48       200

Random Forest accuracy:  0.54


In [31]:
y_new_pred = classifier_poly.predict(X_new)

# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_new,y_new_pred)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_new,y_new_pred)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_new, y_new_pred),4))

Confusion Matrix:
[[25 79]
 [14 82]]

Report model:
              precision    recall  f1-score   support

           0       0.64      0.24      0.35       104
           1       0.51      0.85      0.64        96

    accuracy                           0.54       200
   macro avg       0.58      0.55      0.49       200
weighted avg       0.58      0.54      0.49       200

Random Forest accuracy:  0.535


In [32]:
y_new_pred = classifier_rbf.predict(X_new)

# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_new,y_new_pred)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_new,y_new_pred)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_new, y_new_pred),4))

Confusion Matrix:
[[39 65]
 [22 74]]

Report model:
              precision    recall  f1-score   support

           0       0.64      0.38      0.47       104
           1       0.53      0.77      0.63        96

    accuracy                           0.56       200
   macro avg       0.59      0.57      0.55       200
weighted avg       0.59      0.56      0.55       200

Random Forest accuracy:  0.565


# **KẾT HỢP**

In [34]:
data

,Negative,Positive,return_bitcoin,return_litecoin,avg_diff,act_address,avg_gas,trans_count,block_count,block_size,etherium
Time,,,,,,,,,,,
2017-11-10,0,0,-0.073554,-0.077946,1430000000000000,233966,0.1911,457427,6300,84500680,0.051555
2017-11-11,0,1,-0.039368,0.051353,1440000000000000,194696,0.1867,394378,6267,79370301,-0.021523
2017-11-12,0,0,-0.064101,-0.052933,1490000000000000,213315,0.2106,434380,6245,82639583,0.028606
2017-11-13,0,1,0.102422,0.040523,1460000000000000,214384,0.2203,443176,6225,89907928,0.066037
2017-11-14,0,0,0.011626,0.020899,1450000000000000,226170,0.2359,475984,6122,99374433,-0.012659
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-07,0,1,-0.014124,-0.036100,11088900000000000,517421,2.7500,972121,7075,550842346,0.039498
2022-12-08,1,0,0.022872,0.017574,11088900000000000,528927,2.7300,923887,7161,512401884,-0.013138
2022-12-09,0,1,-0.005821,-0.026219,11088900000000000,1505652,2.3900,1932711,7152,585484918,0.001661


In [37]:
from sklearn.ensemble import RandomForestRegressor

In [42]:
return_bitcoin = pd.DataFrame(data["return_bitcoin"])
return_bitcoin["return_bitcoin_lag"] = return_bitcoin["return_bitcoin"].shift(-1)
return_bitcoin.dropna(inplace=True)

X_set_1 = return_bitcoin.return_bitcoin
y_set_1 = return_bitcoin.return_bitcoin_lag 

X_set_1 = np.array(X_set_1).reshape(-1, 1)

classifier_rf_1 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_1.fit(X_set_1, y_set_1.ravel())
y_set_1_pred = classifier_rf_1.predict(X_set_1)

pd.DataFrame({"New":y_set_1_pred,
              "Real":y_set_1})

,New,Real
Time,,
2017-11-10,-0.024647,-0.039368
2017-11-11,-0.028454,-0.064101
2017-11-12,0.071956,0.102422
2017-11-13,0.025457,0.011626
2017-11-14,0.060534,0.102444
...,...,...
2022-12-06,-0.021870,-0.014124
2022-12-07,0.010543,0.022872
2022-12-08,0.001302,-0.005821


In [44]:
return_bitcoin = pd.DataFrame(data["return_bitcoin"])
return_bitcoin["return_bitcoin_lag"] = return_bitcoin["return_bitcoin"].shift(-1)
return_bitcoin.dropna(inplace=True)

X_set_1 = return_bitcoin.return_bitcoin
y_set_1 = return_bitcoin.return_bitcoin_lag 

X_set_1 = np.array(X_set_1).reshape(-1, 1)

classifier_rf_1 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_1.fit(X_set_1, y_set_1.ravel())
y_set_1_pred = classifier_rf_1.predict(X_set_1)

pd.DataFrame({"New":y_set_1_pred,
              "Real":y_set_1})

,New,Real
Time,,
2017-11-10,-0.024647,-0.039368
2017-11-11,-0.028454,-0.064101
2017-11-12,0.071956,0.102422
2017-11-13,0.025457,0.011626
2017-11-14,0.060534,0.102444
...,...,...
2022-12-06,-0.021870,-0.014124
2022-12-07,0.010543,0.022872
2022-12-08,0.001302,-0.005821


In [45]:
return_litecoin = pd.DataFrame(data["return_litecoin"])
return_litecoin["return_litecoin_lag"] = return_litecoin["return_litecoin"].shift(-1)
return_litecoin.dropna(inplace=True)

X_set_2 = return_litecoin.return_litecoin
y_set_2 = return_litecoin.return_litecoin_lag 

X_set_2 = np.array(X_set_2).reshape(-1, 1)

classifier_rf_2 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_2.fit(X_set_2, y_set_2.ravel())
y_set_2_pred = classifier_rf_2.predict(X_set_2)

pd.DataFrame({"New":y_set_2_pred,
              "Real":y_set_2})

,New,Real
Time,,
2017-11-10,0.031531,0.051353
2017-11-11,-0.028962,-0.052933
2017-11-12,0.023258,0.040523
2017-11-13,0.039616,0.020899
2017-11-14,0.002559,0.018252
...,...,...
2022-12-06,-0.027834,-0.036100
2022-12-07,0.032443,0.017574
2022-12-08,-0.027313,-0.026219


In [46]:
avg_diff = pd.DataFrame(data["avg_diff"])
avg_diff["avg_diff_lag"] = avg_diff["avg_diff"].shift(-1)
avg_diff.dropna(inplace=True)

X_set_3 = avg_diff.avg_diff
y_set_3 = avg_diff.avg_diff_lag 

X_set_3 = np.array(X_set_3).reshape(-1, 1)

classifier_rf_3 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_3.fit(X_set_3, y_set_3.ravel())
y_set_3_pred = classifier_rf_3.predict(X_set_3)

pd.DataFrame({"New":y_set_3_pred,
              "Real":y_set_3})

,New,Real
Time,,
2017-11-10,1.434562e+15,1.440000e+15
2017-11-11,1.466653e+15,1.490000e+15
2017-11-12,1.470758e+15,1.460000e+15
2017-11-13,1.482554e+15,1.450000e+15
2017-11-14,1.445270e+15,1.420000e+15
...,...,...
2022-12-06,1.108890e+16,1.108890e+16
2022-12-07,1.108890e+16,1.108890e+16
2022-12-08,1.108890e+16,1.108890e+16


In [47]:
act_address = pd.DataFrame(data["act_address"])
act_address["act_address_lag"] = act_address["act_address"].shift(-1)
act_address.dropna(inplace=True)

X_set_4 = act_address.act_address
y_set_4 = act_address.act_address_lag 

X_set_4 = np.array(X_set_4).reshape(-1, 1)

classifier_rf_4 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_4.fit(X_set_4, y_set_4.ravel())
y_set_4_pred = classifier_rf_4.predict(X_set_4)

pd.DataFrame({"New":y_set_4_pred,
              "Real":y_set_4})

,New,Real
Time,,
2017-11-10,210901.148,194696.0
2017-11-11,217322.340,213315.0
2017-11-12,214541.520,214384.0
2017-11-13,221258.628,226170.0
2017-11-14,251580.878,258219.0
...,...,...
2022-12-06,524270.558,517421.0
2022-12-07,522856.592,528927.0
2022-12-08,1159355.122,1505652.0


In [48]:
avg_gas = pd.DataFrame(data["avg_gas"])
avg_gas["avg_gas_lag"] = avg_gas["avg_gas"].shift(-1)
avg_gas.dropna(inplace=True)

X_set_5 = avg_gas.avg_gas
y_set_5 = avg_gas.avg_gas_lag 

X_set_5 = np.array(X_set_5).reshape(-1, 1)

classifier_rf_5 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_5.fit(X_set_5, y_set_5.ravel())
y_set_5_pred = classifier_rf_5.predict(X_set_5)

pd.DataFrame({"New":y_set_5_pred,
              "Real":y_set_5})

,New,Real
Time,,
2017-11-10,0.177463,0.1867
2017-11-11,0.176787,0.2106
2017-11-12,0.257734,0.2203
2017-11-13,0.229601,0.2359
2017-11-14,0.217050,0.2525
...,...,...
2022-12-06,2.267284,2.7500
2022-12-07,2.662049,2.7300
2022-12-08,2.233232,2.3900


In [49]:
trans_count = pd.DataFrame(data["trans_count"])
trans_count["trans_count_lag"] = trans_count["trans_count"].shift(-1)
trans_count.dropna(inplace=True)

X_set_6 = trans_count.trans_count
y_set_6 = trans_count.trans_count_lag 

X_set_6 = np.array(X_set_6).reshape(-1, 1)

classifier_rf_6 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_6.fit(X_set_6, y_set_6.ravel())
y_set_6_pred = classifier_rf_6.predict(X_set_6)

pd.DataFrame({"New":y_set_6_pred,
              "Real":y_set_6})

,New,Real
Time,,
2017-11-10,4.407199e+05,394378.0
2017-11-11,4.310705e+05,434380.0
2017-11-12,4.523321e+05,443176.0
2017-11-13,5.206090e+05,475984.0
2017-11-14,5.350381e+05,540593.0
...,...,...
2022-12-06,1.000104e+06,972121.0
2022-12-07,9.334543e+05,923887.0
2022-12-08,1.574954e+06,1932711.0


In [50]:
block_count = pd.DataFrame(data["block_count"])
block_count["block_count_lag"] = block_count["block_count"].shift(-1)
block_count.dropna(inplace=True)

X_set_7 = block_count.block_count
y_set_7 = block_count.block_count_lag 

X_set_7 = np.array(X_set_7).reshape(-1, 1)

classifier_rf_7 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_7.fit(X_set_7, y_set_7.ravel())
y_set_7_pred = classifier_rf_7.predict(X_set_7)

pd.DataFrame({"New":y_set_7_pred,
              "Real":y_set_7})

,New,Real
Time,,
2017-11-10,6348.044526,6267.0
2017-11-11,6235.133969,6245.0
2017-11-12,6219.527367,6225.0
2017-11-13,6203.785333,6122.0
2017-11-14,6183.044079,6237.0
...,...,...
2022-12-06,7105.774161,7075.0
2022-12-07,7152.919167,7161.0
2022-12-08,7147.406854,7152.0


In [51]:
block_size = pd.DataFrame(data["block_size"])
block_size["block_size_lag"] = block_size["block_size"].shift(-1)
block_size.dropna(inplace=True)

X_set_8 = block_size.block_size
y_set_8 = block_size.block_size_lag 

X_set_8 = np.array(X_set_8).reshape(-1, 1)

classifier_rf_8 = RandomForestRegressor(n_estimators = 500, random_state =n_state)
classifier_rf_8.fit(X_set_8, y_set_8.ravel())
y_set_8_pred = classifier_rf_8.predict(X_set_8)

pd.DataFrame({"New":y_set_8_pred,
              "Real":y_set_8})

,New,Real
Time,,
2017-11-10,8.162545e+07,79370301.0
2017-11-11,8.623339e+07,82639583.0
2017-11-12,8.799975e+07,89907928.0
2017-11-13,1.025849e+08,99374433.0
2017-11-14,1.088146e+08,102617919.0
...,...,...
2022-12-06,5.430360e+08,550842346.0
2022-12-07,5.165964e+08,512401884.0
2022-12-08,5.642618e+08,585484918.0


In [52]:
data.columns

Index(['Negative', 'Positive', 'return_bitcoin', 'return_litecoin', 'avg_diff',
       'act_address', 'avg_gas', 'trans_count', 'block_count', 'block_size',
       'etherium'],
      dtype='object')

In [53]:
df_concat = pd.DataFrame({"return_bitcoin":y_set_1_pred,
                          "return_litecoin":y_set_2_pred,
                          "avg_diff":y_set_3_pred,
                          "act_address":y_set_4_pred,
                          "avg_gas":y_set_5_pred,
                          "trans_count":y_set_6_pred,
                          "block_count":y_set_7_pred,
                          "block_size":y_set_8_pred},index=y_set_1.index)

In [54]:
df_concat

,return_bitcoin,return_litecoin,avg_diff,act_address,avg_gas,trans_count,block_count,block_size
Time,,,,,,,,
2017-11-10,-0.024647,0.031531,1.434562e+15,210901.148,0.177463,4.407199e+05,6348.044526,8.162545e+07
2017-11-11,-0.028454,-0.028962,1.466653e+15,217322.340,0.176787,4.310705e+05,6235.133969,8.623339e+07
2017-11-12,0.071956,0.023258,1.470758e+15,214541.520,0.257734,4.523321e+05,6219.527367,8.799975e+07
2017-11-13,0.025457,0.039616,1.482554e+15,221258.628,0.229601,5.206090e+05,6203.785333,1.025849e+08
2017-11-14,0.060534,0.002559,1.445270e+15,251580.878,0.217050,5.350381e+05,6183.044079,1.088146e+08
...,...,...,...,...,...,...,...,...
2022-12-06,-0.021870,-0.027834,1.108890e+16,524270.558,2.267284,1.000104e+06,7105.774161,5.430360e+08
2022-12-07,0.010543,0.032443,1.108890e+16,522856.592,2.662049,9.334543e+05,7152.919167,5.165964e+08
2022-12-08,0.001302,-0.027313,1.108890e+16,1159355.122,2.233232,1.574954e+06,7147.406854,5.642618e+08


In [59]:
news = df[["Negative","Positive","return_etherium"]]
news

,Negative,Positive,return_etherium
Train sheet,,,
2017-11-10,0,0,1
2017-11-11,0,1,0
2017-11-12,0,0,1
2017-11-13,0,1,1
2017-11-14,0,0,0
...,...,...,...
2022-12-07,0,1,1
2022-12-08,1,0,0
2022-12-09,0,1,1


# Xử lý data

In [60]:
df_done = pd.concat([df_concat,news],axis=1)
df_done.dropna(axis=0,inplace=True)
df_done

,return_bitcoin,return_litecoin,avg_diff,act_address,avg_gas,trans_count,block_count,block_size,Negative,Positive,return_etherium
2017-11-10,-0.024647,0.031531,1.434562e+15,210901.148,0.177463,4.407199e+05,6348.044526,8.162545e+07,0,0,1
2017-11-11,-0.028454,-0.028962,1.466653e+15,217322.340,0.176787,4.310705e+05,6235.133969,8.623339e+07,0,1,0
2017-11-12,0.071956,0.023258,1.470758e+15,214541.520,0.257734,4.523321e+05,6219.527367,8.799975e+07,0,0,1
2017-11-13,0.025457,0.039616,1.482554e+15,221258.628,0.229601,5.206090e+05,6203.785333,1.025849e+08,0,1,1
2017-11-14,0.060534,0.002559,1.445270e+15,251580.878,0.217050,5.350381e+05,6183.044079,1.088146e+08,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-06,-0.021870,-0.027834,1.108890e+16,524270.558,2.267284,1.000104e+06,7105.774161,5.430360e+08,0,1,0
2022-12-07,0.010543,0.032443,1.108890e+16,522856.592,2.662049,9.334543e+05,7152.919167,5.165964e+08,0,1,1
2022-12-08,0.001302,-0.027313,1.108890e+16,1159355.122,2.233232,1.574954e+06,7147.406854,5.642618e+08,1,0,0
2022-12-09,0.004236,0.008354,1.108890e+16,709505.392,2.406320,1.324169e+06,7159.877022,5.268081e+08,0,1,1


In [71]:
df_new_sub = df_done.loc['2021-11-11':'2022-05-29']
df_done = df_done.loc['2017-11-10':'2021-11-10']


In [72]:
# Set the independent variable property
feature_sub = df_done.columns[df_done.columns!='return_etherium'].to_list()
target_sub =  [df_done.columns[-1]]

y_sub = df_done[target_sub]
X_sub = df_done[feature_sub]

In [73]:
# Split the train and test
n_state = 45
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X_sub,y_sub, train_size = 0.8, test_size = 0.2, random_state=n_state)

In [74]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Call scale function
# sc_X = MinMaxScaler(feature_range=(0, 1))
sc_X_sub = StandardScaler()
# Scale train set
X_train_sub = sc_X_sub.fit_transform(X_train_sub)
# Scale test set
X_test_sub = sc_X_sub.fit_transform(X_test_sub)

# Mô hình mới

In [75]:
classifier_poly_sub = SVC(kernel = 'poly', probability=True, random_state = 45)
classifier_poly_sub.fit(X_train_sub, y_train_sub)

y_pred_sub = classifier_poly_sub.predict(X_test_sub)

print('Confusion matrix:')
print(confusion_matrix(y_test_sub,y_pred_sub),'\n')

print('Classification report:')
print(classification_report(y_test_sub,y_pred_sub))

print('Support Vector Machine accuracy: ', round(accuracy_score(y_test_sub, y_pred_sub),4))

Confusion matrix:
[[ 88  57]
 [  8 140]] 

Classification report:
              precision    recall  f1-score   support

           0       0.92      0.61      0.73       145
           1       0.71      0.95      0.81       148

    accuracy                           0.78       293
   macro avg       0.81      0.78      0.77       293
weighted avg       0.81      0.78      0.77       293

Support Vector Machine accuracy:  0.7782


In [76]:
classifier_lr_sub = SVC(kernel = 'linear', probability=True, random_state = 45)
classifier_lr_sub.fit(X_train_sub, y_train_sub)

y_pred_sub = classifier_lr_sub.predict(X_test_sub)

print('Confusion matrix:')
print(confusion_matrix(y_test_sub,y_pred_sub),'\n')

print('Classification report:')
print(classification_report(y_test_sub,y_pred_sub))

print('Support Vector Machine accuracy: ', round(accuracy_score(y_test_sub, y_pred_sub),4))

Confusion matrix:
[[108  37]
 [ 19 129]] 

Classification report:
              precision    recall  f1-score   support

           0       0.85      0.74      0.79       145
           1       0.78      0.87      0.82       148

    accuracy                           0.81       293
   macro avg       0.81      0.81      0.81       293
weighted avg       0.81      0.81      0.81       293

Support Vector Machine accuracy:  0.8089


In [77]:
classifier_rbf_sub = SVC(kernel = 'rbf', probability=True, random_state = 45)
classifier_rbf_sub.fit(X_train_sub, y_train_sub)

y_pred_sub = classifier_rbf_sub.predict(X_test_sub)

print('Confusion matrix:')
print(confusion_matrix(y_test_sub,y_pred_sub),'\n')

print('Classification report:')
print(classification_report(y_test_sub,y_pred_sub))

print('Support Vector Machine accuracy: ', round(accuracy_score(y_test_sub, y_pred_sub),4))

Confusion matrix:
[[106  39]
 [ 18 130]] 

Classification report:
              precision    recall  f1-score   support

           0       0.85      0.73      0.79       145
           1       0.77      0.88      0.82       148

    accuracy                           0.81       293
   macro avg       0.81      0.80      0.80       293
weighted avg       0.81      0.81      0.80       293

Support Vector Machine accuracy:  0.8055


# KIỂM ĐỊNH MÔ HÌNH MỚI TRÊN DATASET MỚI

In [ ]:
# Set the independent variable property
feature_new_sub = df_new_sub.columns[df_new_sub.columns!='return_etherium'].to_list()
target_new_sub =  [df_new_sub.columns[-1]]

y_new_sub = df_new_sub[target_new_sub]
X_new_sub = df_new_sub[feature_new_sub]

sc_X_new_sub = StandardScaler()

# Scale train set
X_new_sub = sc_X_new_sub.fit_transform(X_new_sub)

In [79]:
y_new_pred_sub = classifier_rbf_sub.predict(X_new_sub)

# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_new_sub,y_new_pred_sub)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_new_sub,y_new_pred_sub)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_new_sub,y_new_pred_sub),4))

Confusion Matrix:
[[  0 104]
 [  0  96]]

Report model:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       104
           1       0.48      1.00      0.65        96

    accuracy                           0.48       200
   macro avg       0.24      0.50      0.32       200
weighted avg       0.23      0.48      0.31       200

Random Forest accuracy:  0.48


In [80]:
y_new_pred_sub = classifier_poly_sub.predict(X_new_sub)

# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_new_sub,y_new_pred_sub)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_new_sub,y_new_pred_sub)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_new_sub,y_new_pred_sub),4))

Confusion Matrix:
[[104   0]
 [ 96   0]]

Report model:
              precision    recall  f1-score   support

           0       0.52      1.00      0.68       104
           1       0.00      0.00      0.00        96

    accuracy                           0.52       200
   macro avg       0.26      0.50      0.34       200
weighted avg       0.27      0.52      0.36       200

Random Forest accuracy:  0.52


In [82]:
y_new_pred_sub = classifier_lr_sub.predict(X_new_sub)

# Ma trận bối rối
print(f'Confusion Matrix:\n{confusion_matrix(y_new_sub,y_new_pred_sub)}\n')

# Báo cáo phân loại
print(f'Report model:\n{classification_report(y_new_sub,y_new_pred_sub)}')

# Tỉ lệ chính xác accuracy
print('Random Forest accuracy: ', round(accuracy_score(y_new_sub,y_new_pred_sub),4))

Confusion Matrix:
[[104   0]
 [ 96   0]]

Report model:
              precision    recall  f1-score   support

           0       0.52      1.00      0.68       104
           1       0.00      0.00      0.00        96

    accuracy                           0.52       200
   macro avg       0.26      0.50      0.34       200
weighted avg       0.27      0.52      0.36       200

Random Forest accuracy:  0.52
